# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
lr_model = LinearRegression()
# Calculate a baseline r-squared score on training data
lr_model.fit(X_train_scaled, y_train)
r2_train = lr_model.score(X_train_scaled, y_train)

r2_train

0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure
scores = {}
combs = list(combinations(X_train.columns, 2))
combs
# Find combinations of columns and loop over them
for comb in combs:
    # Make copies of X_train and X_val
    X_train_copy = X_train.copy()
    X_val_copy = X_val.copy()
    
    # Add interaction term to data
    X_train_copy['interactions'] = X_train_copy[comb[0]] * X_train_copy[comb[1]]
    X_val_copy['interactions'] = X_val_copy[comb[0]] * X_val_copy[comb[1]]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    lr_model.fit(X_train_copy, y_train)
    #lr_model.score(X_val_copy, y_val)
    
    # Append to data structure
    scores[comb] = lr_model.score(X_val_copy, y_val)
    
# Sort and subset the data structure to find the top 7
print(list(sorted(scores.items(), key = lambda x : x[1], reverse = True))[0:7])

[(('LotArea', '1stFlrSF'), 0.7211105666140566), (('LotArea', 'TotalBsmtSF'), 0.7071649207050104), (('LotArea', 'GrLivArea'), 0.6690980823779031), (('LotArea', 'Fireplaces'), 0.6529699515652584), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.6472994890405191), (('OverallCond', 'TotalBsmtSF'), 0.6429019879233768), (('OverallQual', '2ndFlrSF'), 0.6422324294284368)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Your code here
for key, val in list(sorted(scores.items(), key = lambda x : x[1], reverse= True))[0:7]:
    #print(key[0], val)
    X_train[key[0] + "*" + key[1]] = X_train[key[0]] * X_train[key[1]]
    X_val[key[0] + "*" + key[1]] = X_val[key[0]] * X_val[key[1]]
X_train.head()
# Loop over top 7 interactions

    # Extract column names from data structure

    # Construct new column name
    
    # Add new column to X_train and X_val


,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea*1stFlrSF,LotArea*TotalBsmtSF,LotArea*GrLivArea,LotArea*Fireplaces,2ndFlrSF*TotRmsAbvGrd,OverallCond*TotalBsmtSF,OverallQual*2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799


In [7]:
X_val

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea*1stFlrSF,LotArea*TotalBsmtSF,LotArea*GrLivArea,LotArea*Fireplaces,2ndFlrSF*TotRmsAbvGrd,OverallCond*TotalBsmtSF,OverallQual*2ndFlrSF
0,-0.491264,-0.099842,0.397681,-0.248487,-0.598161,-0.808132,-1.138195,-0.981739,-0.178895,-0.994820,0.293855,0.122073,0.559154,0.488719,0.793375,-0.098818,0.080686
1,-0.619291,0.632038,-0.509252,-0.205591,-0.549222,0.849426,0.327177,0.250689,-0.465607,-0.994820,0.340128,0.127321,-0.202618,0.616083,0.212942,0.104698,0.536870
2,0.240165,-0.099842,0.397681,-0.639316,-1.044043,0.722619,-0.137834,-0.365525,-0.427379,-0.994820,-0.250742,-0.153541,-0.033103,-0.238921,-0.264135,-0.254244,-0.072148
3,0.037695,2.095798,-3.230050,1.891539,1.843314,-0.808132,0.616344,0.866903,1.703847,0.588023,0.069483,0.071301,0.023233,0.022165,-0.700572,-6.109766,-1.693681
4,-0.147924,0.632038,-0.509252,-1.616388,-0.995104,0.709032,-0.114388,0.250689,-0.408265,0.588023,0.147199,0.239102,0.016921,-0.086983,0.177746,0.823149,0.448135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,-0.285276,-1.563603,-0.509252,-0.653614,-1.060355,-0.808132,-1.470346,-1.597953,-2.281450,2.170867,0.302494,0.186461,0.419455,-0.619297,1.291357,0.332855,1.263597
270,-0.029714,1.363918,2.211546,-0.498713,1.008645,1.859359,2.318130,2.099330,-0.178895,2.170867,-0.029971,0.014819,-0.068880,-0.064505,3.903410,-1.102926,2.536014
271,-0.741509,-0.099842,0.397681,-1.380461,-1.889587,0.333138,-1.081534,-0.981739,-0.943461,-0.994820,1.401146,1.023624,0.801967,0.737668,-0.327054,-0.548982,-0.033261
272,-0.147106,-0.099842,-0.509252,1.476879,1.419183,-0.808132,0.311547,0.250689,0.318072,-0.994820,-0.208770,-0.217257,-0.045830,0.146344,-0.202590,-0.752104,0.080686


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [8]:
# Your code here
poly_scores = []
for col in X_train.columns:
    for i in range(2,5):
        #print(i)
        X_train_copy = X_train.copy()
        X_val_copy = X_val.copy()
        
        poly = PolynomialFeatures(degree = i, include_bias = False)
        
        X_train_poly = pd.DataFrame(poly.fit_transform(X_train_copy[[col]]))
        X_train_copy.drop(col, inplace = True, axis = 1)
        X_train_copy = pd.concat([X_train_copy, X_train_poly], axis = 1)
        
        
        X_val_poly = pd.DataFrame(poly.transform(X_val_copy[[col]]))
        X_val_copy.drop(col, inplace = True, axis = 1)
        X_val_copy = pd.concat([X_val_copy, X_val_poly], axis = 1)
        lr = LinearRegression()
        lr.fit(X_train_copy, y_train)
    
        poly_scores.append((col, i, lr.score(X_val_copy, y_val)))
print("Top 7 scores: ", sorted(poly_scores, key = lambda x: x[2], reverse = True)[:7])
# Set up data structure

# Loop over all columns

    # Loop over degrees 2, 3, 4
        
        # Make a copy of X_train and X_val
    
        # Instantiate PolynomialFeatures with relevant degree
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
    
        # Find r-squared score on validation
    
        # Append to data structure

# Sort and subset the data structure to find the top 7


Top 7 scores:  [('GrLivArea', 3, 0.8283344365591127), ('OverallQual*2ndFlrSF', 3, 0.8222074062501777), ('LotArea*Fireplaces', 4, 0.8126598088712166), ('LotArea*Fireplaces', 3, 0.8124111133929883), ('OverallQual', 3, 0.8068023565969222), ('OverallQual*2ndFlrSF', 2, 0.8057607786926371), ('OverallQual', 4, 0.8033455378866854)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [9]:
top_polynomials = pd.DataFrame(sorted(poly_scores, key = lambda x: x[2], reverse = True)[:7], columns=["Column", "Degree", "R^2"])
top_polynomials

# Drop duplicate columns based on Column name
top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials

# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    #Create polynomimal terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                              columns=poly.get_feature_names([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                              columns=poly.get_feature_names([col]))
    
    #Concat back to original
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [10]:
# Your code here
X_train.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea*1stFlrSF,LotArea*TotalBsmtSF,...,OverallQual*2ndFlrSF,OverallQual*2ndFlrSF^2,OverallQual*2ndFlrSF^3,LotArea*Fireplaces,LotArea*Fireplaces^2,LotArea*Fireplaces^3,LotArea*Fireplaces^4,OverallQual,OverallQual^2,OverallQual^3
0,-0.114710,-0.509252,-0.639316,-0.804789,1.261552,0.250689,0.327629,-0.994820,0.092318,0.073336,...,-0.125956,0.015865,-0.001998,0.114116,0.013022,0.001486,1.695855e-04,-0.099842,0.009968,-0.000995
1,-0.176719,-0.509252,0.838208,0.641608,-0.808132,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,...,-0.510770,0.260886,-0.133253,0.175804,0.030907,0.005434,9.552459e-04,0.632038,0.399472,0.252481
2,-0.246336,1.304613,-0.012560,-0.329000,-0.808132,-0.981739,-1.105931,-0.994820,0.081045,0.003094,...,0.672141,0.451774,0.303656,0.245060,0.060055,0.014717,3.606557e-03,-0.831723,0.691762,-0.575354
3,-0.378617,1.304613,-0.339045,-0.609036,-0.808132,-0.981739,-1.134602,0.588023,0.230591,0.128368,...,0.672141,0.451774,0.303656,-0.222636,0.049567,-0.011035,2.456852e-03,-0.831723,0.691762,-0.575354
4,-0.010898,1.304613,-2.531499,-1.315922,0.550523,0.250689,-2.281450,-0.994820,0.014341,0.027589,...,-0.860799,0.740974,-0.637829,0.010842,0.000118,0.000001,1.381725e-08,-1.563603,2.444854,-3.822780


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [11]:
# Your code here
lr_model.fit(X_train, y_train)
print(lr_model.score(X_train, y_train))
print(lr_model.score(X_val, y_val))

0.8571817758242435
0.6442143449159523


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [12]:
# Your code here
for i in [5, 10, 15, 20]:
    
    rfe = RFE(LinearRegression(), n_features_to_select=i)
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_val_rfe = rfe.transform(X_val)

    lr = LinearRegression()
    lr.fit(X_train_rfe, y_train)

    print("Train score:", lr.score(X_train_rfe, y_train))
    print("Val score: ", lr.score(X_val_rfe, y_val))
    print("Num features and col: ", i, len(X_train.columns))

Train score: 0.776039994126505
Val score:  0.6352981725272362
Num features and col:  5 26
Train score: 0.8191862278324273
Val score:  0.6743476159860744
Num features and col:  10 26
Train score: 0.8483321237427194
Val score:  0.7040137671087112
Num features and col:  15 26
Train score: 0.8495176468836854
Val score:  0.7169477986870836
Num features and col:  20 26


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [13]:
# Your code here
for a in [10, 100, 1000]:
    las = Lasso(alpha =a)
    las.fit(X_train, y_train)
    
    print("Train score:", las.score(X_train, y_train))
    print("Val score: ", las.score(X_val, y_val))
    print(" Alpha: ", a, "\n")
    #print("Num features and col: ", i, len(X_train.columns))

Train score: 0.8571373079024015
Val score:  0.6480527180183087
 Alpha:  10 

Train score: 0.856958744623801
Val score:  0.6471042867008591
 Alpha:  100 

Train score: 0.8506404012942794
Val score:  0.7222278677869786
 Alpha:  1000 



Compare the results. Which features would you choose to use?

In [14]:
# Your written answer here
#I would choose lasso with alpha = 1000 because it has the highest val score
#For RFE I would choose the 20 columns chosen in the last iteration. It has the highest val score

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [15]:
# Your code here
X_test_scaled = scaler.transform(X_test)

X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

for key, val in list(sorted(scores.items(), key = lambda x : x[1], reverse= True))[0:7]:
    #print(key[0], val)
    X_test[key[0] + "*" + key[1]] = X_test[key[0]] * X_test[key[1]]
    #X_val[key[0] + "*" + key[1]] = X_val[key[0]] * X_val[key[1]]
    
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [19]:
# Your code here

final = Lasso(alpha =1000)
final.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("Test score: ", final.score(X_test, y_test))
print(" Alpha: ", a, "\n")
print("MSE: ", mean_squared_error(y_test, final.predict(X_test)))

Test score:  0.8537550456513499
 Alpha:  1000 

MSE:  1024491102.5575491


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.